## Datlinq data assessment
The first step is to import the the packages I'll be using. I'll start with Pandas for importing the data and initial cleaning of the data. Various parts of Scikit-Learn will become important in the next steps of processing the data.

In [9]:
import pandas as pd
from src.functions import import_json_to_df

### Data preparation
The first step is to import the data into Pandas to prepare it for processing. The function `import_json_to_df` uses `json_normalize` to flatten nested dicts inside the JSON into new columns. 

In [10]:
facebook_path = "data-sample/facebook-rotterdam-20170131.json"
factual_path = "data-sample/factual-rotterdam-20170207.csv"
google_path = "data-sample/google-rotterdam-20170207.json"

df_facebook = import_json_to_df(facebook_path)
df_factual = pd.read_csv(factual_path)
df_google = import_json_to_df(google_path)

In [8]:
df_facebook

,__location,__reference,__timestamp,__timestamp_basic,__timestamp_detail,__timestamp_insights,__timestamp_photos,_id,about,affiliation,...,products,public_transit,release_date,restaurant_services,restaurant_specialties,store_number,studio,talking_about_count,website,were_here_count
0,"{'type': 'Point', 'coordinates': [4.4890279881...",NaN,{'$numberLong': '1483522918'},{'$numberLong': '1483522918'},NaN,NaN,NaN,362980210726552,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"{'type': 'Point', 'coordinates': [4.4872103297...",NaN,{'$numberLong': '1483522918'},{'$numberLong': '1483522918'},NaN,NaN,NaN,159873390861865,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,"{'type': 'Point', 'coordinates': [4.4752998, 5...",{'3012AR17': '3012AR17'},{'$numberLong': '1444725009'},{'$numberLong': '1444725009'},NaN,NaN,NaN,785657301493030,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,"{'type': 'Point', 'coordinates': [4.4883539, 5...",NaN,{'$numberLong': '1483522918'},{'$numberLong': '1483522918'},NaN,NaN,NaN,1705013096377668,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"{'type': 'Point', 'coordinates': [4.4340983823...",NaN,{'$numberLong': '1483930996'},{'$numberLong': '1442919593'},{'$numberLong': '1483930996'},NaN,NaN,596377147166585,Welkom op de actie pagina van Rotterdam in Act...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'$numberLong': '1'},http://www.vakbondinactie.nl,{'$numberLong': '0'}
5,"{'type': 'Point', 'coordinates': [4.4765499, 5...",{'1257941': '1257941'},{'$numberLong': '1484001628'},{'$numberLong': '1482266004'},{'$numberLong': '1484001628'},NaN,NaN,8146619641,We are a strategically led digital agency that...,NaN,...,"Digital Strategy, Customer Experience, Website...",\n,NaN,NaN,NaN,NaN,NaN,{'$numberLong': '8'},http://mangrove.com,{'$numberLong': '1559'}
6,"{'type': 'Point', 'coordinates': [4.4938647445...",{'3083CN10912': '3083CN10912'},{'$numberLong': '1483339037'},{'$numberLong': '1439056464'},{'$numberLong': '1483339037'},{'$numberLong': '1439827948'},{'$numberLong': '1439827946'},137171206389676,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'$numberLong': '0'},NaN,{'$numberLong': '107'}
7,"{'type': 'Point', 'coordinates': [4.4768141, 5...",{'1257941': '1257941'},{'$numberLong': '1483609898'},{'$numberLong': '1482266004'},{'$numberLong': '1483609898'},NaN,NaN,204661816581880,the 30th of April we're hosting our first club...,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'$numberLong': '0'},NaN,{'$numberLong': '10'}
8,"{'type': 'Point', 'coordinates': [4.45801, 51....",{'1193342': '1193342'},{'$numberLong': '1483375257'},{'$numberLong': '1482261562'},{'$numberLong': '1483375257'},NaN,NaN,1441185105897654,Appartementen Verhuur en Kamers,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,{'$numberLong': '0'},NaN,{'$numberLong': '0'}
9,"{'type': 'Point', 'coordinates': [4.48659, 51....",{'309185': '309185'},{'$numberLong': '1483622023'},{'$numberLong': '1482250260'},{'$numberLong': '1483622023'},{'$numberLong': '1440760587'},{'$numberLong': '1440760586'},211111138928095,In hartje Rotterdam vind u De Smaak van Afrika...,NaN,...,NaN,Tram 7 & 8 halte meent.,NaN,"{'delivery': {'$numberLong': '0'}, 'catering':...","{'breakfast': {'$numberLong': '0'}, 'coffee': ...",NaN,NaN,{'$numberLong': '10'},desmaakvanafrika.nl,{'$numberLong': '1964'}
